In [1]:
from cot import Collection
from cot.stats import evaluation_as_table
from numpy import loadtxt

In [70]:
coll = Collection.load_thoughtsource_100()

In [71]:
# select just our own created COTs
coll.select_generated_cots(author="thoughtsource")

In [72]:
# to get the answer_from_choices, we need to evaluate with overwrite=True for now
coll.evaluate(overwrite=True)
table = evaluation_as_table(eval)
table

/home/kon/work/ThoughtSource/libs/cot/cot/stats.py:406: PerformanceWarning: indexing past lexsort depth may impact performance.
  df.loc[dataset, (instruction + "_" + cot_trigger, model)] = v


In [82]:
# get the answer_from_choices, here just from the first of the generated COTs as an example
# this function is way to slow...
# we just use med_qa for now
answer_from_choices = []
for i in range(len(coll["med_qa"]["test"]["generated_cot"])):
    answer_from_choices.append(coll["med_qa"]["test"]["generated_cot"][i][0]["answers"][0]["answer_from_choices"])

In [83]:
print(answer_from_choices)

['D', 'C', 'D', 'C', 'D', 'D', 'D', 'D', 'D', 'E', 'C', 'C', 'B', 'A', 'A', 'C', 'C', 'B', 'E', 'B', 'D', 'D', 'A', 'E', 'E', 'B', 'C', 'B', 'D', 'E', 'D', 'A', 'A', 'C', 'A', 'D', 'D', 'D', 'E', 'D', 'A', 'D', 'B', 'C', 'D', 'D', '', 'E', 'D', 'A', 'B', 'B', 'A', 'B', 'D', 'E', 'D', 'B', 'D', '', 'C', 'D', 'D', '', 'B', 'E', 'D', '', 'D', 'D', 'D', 'B', 'E', '', 'A', 'D', 'B', 'E', 'D', 'D', 'A', 'D', 'C', 'D', 'E', '', 'E', 'C', 'C', 'D', 'D', 'A', 'B', '', 'A', 'B', 'C', 'B', 'B', '']


In [84]:
# get correct answer choices from the each item of a dataset
def find_correct_choice(lists1, lists2):
    output = []
    for list1, list2 in zip(lists1, lists2):
        sub_output = []
        for item in list2:
            position = list1.index(item) + 1
            sub_output.append(chr(ord('A') + position - 1))
        output.extend(sub_output)
    return output

In [85]:
correct_choices = find_correct_choice(coll["med_qa"]["test"]["choices"], coll["med_qa"]["test"]["answer"])
print(correct_choices)

['C', 'D', 'C', 'C', 'C', 'D', 'A', 'A', 'E', 'A', 'E', 'C', 'B', 'D', 'C', 'D', 'D', 'A', 'C', 'B', 'D', 'D', 'A', 'D', 'C', 'D', 'C', 'B', 'B', 'D', 'D', 'C', 'E', 'C', 'A', 'A', 'D', 'D', 'A', 'B', 'B', 'A', 'B', 'E', 'D', 'C', 'B', 'E', 'C', 'C', 'B', 'B', 'A', 'B', 'B', 'D', 'E', 'C', 'D', 'B', 'A', 'E', 'C', 'B', 'E', 'D', 'E', 'C', 'C', 'E', 'D', 'B', 'C', 'B', 'B', 'B', 'C', 'B', 'C', 'D', 'C', 'A', 'C', 'D', 'A', 'E', 'A', 'A', 'C', 'E', 'B', 'E', 'C', 'B', 'B', 'A', 'A', 'B', 'B', 'C']


In [86]:
import krippendorff

In [87]:
# compute metric
krippendorff.alpha(reliability_data=[answer_from_choices, correct_choices],level_of_measurement='nominal')

0.14325825170025586

In [61]:
coll.unload_datasets(["med_qa"], reverse=True)

In [62]:
coll.dump("ts_100_med_qa_2.json")